In [113]:
from parser_transitions import *
from parser_model import *
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:

def test_dependencies(name, deps, ex_deps):
    """Tests the provided dependencies match the expected dependencies"""
    deps = tuple(sorted(deps))
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)


def test_minibatch_parse():
    """Simple tests for the minibatch_parse function
    Warning: these are not exhaustive
    """

    # Unidirectional arcs test
    sentences = [["right", "arcs", "only"],
                 ["right", "arcs", "only", "again"],
                 ["left", "arcs", "only"],
                 ["left", "arcs", "only", "again"]]
    deps = minibatch_parse(sentences, DummyModel(), 2)
    test_dependencies("minibatch_parse", deps[0],
                      (('ROOT', 'right'), ('arcs', 'only'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[1],
                      (('ROOT', 'right'), ('arcs', 'only'), ('only', 'again'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[2],
                      (('only', 'ROOT'), ('only', 'arcs'), ('only', 'left')))
    test_dependencies("minibatch_parse", deps[3],
                      (('again', 'ROOT'), ('again', 'arcs'), ('again', 'left'), ('again', 'only')))

    # Out-of-bound test
    sentences = [["right"]]
    deps = minibatch_parse(sentences, DummyModel(), 2)
    test_dependencies("minibatch_parse", deps[0], (('ROOT', 'right'),))

    # Mixed arcs test
    sentences = [["this", "is", "interleaving", "dependency", "test"]]
    deps = minibatch_parse(sentences, DummyModel(mode="interleave"), 1)
    test_dependencies("minibatch_parse", deps[0],
                      (('ROOT', 'is'), ('dependency', 'interleaving'),
                      ('dependency', 'test'), ('is', 'dependency'), ('is', 'this')))
    print("minibatch_parse test passed!")


# if __name__ == '__main__':
#     args = sys.argv
#     if len(args) != 2:
#         raise Exception("You did not provide a valid keyword. Either provide 'part_c' or 'part_d', when executing this script")
#     elif args[1] == "part_c":
#         test_parse_step()
#         test_parse()
#     elif args[1] == "part_d":
#         test_minibatch_parse()
#     else:
#         raise Exception("You did not provide a valid keyword. Either provide 'part_c' or 'part_d', when executing this script")


In [48]:
test_parse_step()
test_parse()

SHIFT test passed!
LEFT-ARC test passed!
RIGHT-ARC test passed!
parse test passed!


In [115]:
    embeddings = np.zeros((100, 30), dtype=np.float32)
    model = ParserModel(embeddings)

    def check_embedding():
        inds = torch.randint(0, 100, (4, 36), dtype=torch.long)
        selected = model.embedding_lookup(inds)
        assert np.all(selected.data.numpy() == 0), "The result of embedding lookup: " \
                                      + repr(selected) + " contains non-zero elements."

    def check_forward():
        inputs = torch.randint(0, 100, (4, 36), dtype=torch.long)
        out = model(inputs)
        expected_out_shape = (4, 3)
        assert out.shape == expected_out_shape, "The result shape of forward is: " + repr(out.shape) + \
                                                " which doesn't match expected " + repr(expected_out_shape)

In [122]:
check_embedding()

In [123]:
check_forward()

RuntimeError: 1D tensors expected, got 2D, 2D tensors at /opt/conda/conda-bld/pytorch_1591914855613/work/aten/src/TH/generic/THTensorEvenMoreMath.cpp:431

In [116]:
test_minibatch_parse()

0
['ROOT', 'right'] ['arcs', 'only'] []


1
['ROOT', 'right'] ['arcs', 'only', 'again'] []


2
['ROOT'] ['left', 'arcs', 'only'] []


3
['ROOT'] ['left', 'arcs', 'only', 'again'] []


0
['ROOT', 'right', 'arcs'] ['only'] []


1
['ROOT', 'right', 'arcs'] ['only', 'again'] []


2
['ROOT'] ['left', 'arcs', 'only'] []


3
['ROOT'] ['left', 'arcs', 'only', 'again'] []


0
['ROOT', 'right', 'arcs', 'only'] [] []


1
['ROOT', 'right', 'arcs', 'only'] ['again'] []


2
['ROOT'] ['left', 'arcs', 'only'] []


3
['ROOT'] ['left', 'arcs', 'only', 'again'] []


0
['ROOT', 'right', 'arcs'] [] [('arcs', 'only')]


1
['ROOT', 'right', 'arcs', 'only', 'again'] [] []


2
['ROOT'] ['left', 'arcs', 'only'] []


3
['ROOT'] ['left', 'arcs', 'only', 'again'] []


0
['ROOT', 'right'] [] [('arcs', 'only'), ('right', 'arcs')]


1
['ROOT', 'right', 'arcs', 'only'] [] [('only', 'again')]


2
['ROOT'] ['left', 'arcs', 'only'] []


3
['ROOT'] ['left', 'arcs', 'only', 'again'] []


0
['ROOT'] [] [('arcs', 'only'), ('

IndexError: list assignment index out of range

- Currently we couldn't totally implement the function because we need to implement the right model.

In [60]:
embeddings = torch.tensor(np.zeros((100, 30), dtype=np.float32))

In [54]:
inds = torch.randint(0, 100, (4, 36), dtype=torch.long)

In [61]:
inds

tensor([[48, 27, 86, 82, 82, 22, 38, 37,  2, 53, 99,  1, 78, 47, 33, 26, 25, 29,
         27, 91, 54, 11, 72, 59, 63, 41, 86, 99, 44, 71, 51,  9, 53, 82, 24, 36],
        [ 7, 46, 28, 36, 79, 92, 96, 24, 52, 53, 11, 93, 48, 59, 79, 52, 88, 91,
         38,  5, 79, 19, 73, 72, 55, 74, 10, 63, 27, 30, 58,  9, 90, 85, 96, 56],
        [53, 62, 20, 22, 20, 86, 89, 78, 36, 66, 55, 86, 76, 37, 27, 50, 51, 39,
         90, 60, 36,  4, 21, 26, 51,  6, 22,  8, 99, 29, 58, 78, 97, 25, 19, 23],
        [79,  7, 38, 73, 36, 10, 95, 39, 89, 49, 87, 16, 98, 89, 32, 29,  1, 76,
         96,  9, 26, 66, 85, 21, 74, 72, 44, 50, 61, 82, 81,  3,  2, 52,  5, 32]])

In [74]:
embeddings.shape, inds.shape

(torch.Size([100, 30]), torch.Size([4, 36]))

In [99]:
4, 36*30

(4, 1080)

In [90]:
x1 = torch.index_select(embeddings, 0, inds[0]).flatten()
x2 = torch.index_select(embeddings, 0, inds[1]).flatten()
x3 = torch.index_select(embeddings, 0, inds[2]).flatten()
x4 = torch.index_select(embeddings, 0, inds[3]).flatten()


In [96]:
torch.stack((x1,x2,x3,x4), dim=0).shape

torch.Size([4, 1080])

In [110]:
torch.stack([torch.index_select(embeddings, 0, inds[i]).flatten() for i in range (inds.shape[0])], dim=0)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])